# PART 1

## Data understanding

In [1]:
# JUST for data understanding
import pandas as pd

In [ ]:
police=pd.read_csv("C:/Users/lds/Desktop/ds lab/LDS_Project_2023/Police.csv")

In [ ]:
for item in police:
    unique_values=police[item].unique().tolist()
    print(item, "has ", len(unique_values), "unique values.")

In [ ]:
# Count the number of missing values in each column
missing_values = police.isnull().sum()

# Print the result
print(missing_values)

In [ ]:
police["gun_stolen"].unique()

## csv preparation

In [1]:
import csv
import pyodbc
import json

In [2]:
with open("Police.csv", "r") as file:
    lines=file.readlines()

In [3]:
i = 0

for e in lines[0].split(','):
    print(i, e)
    i += 1

0 custody_id
1 participant_age_group
2 participant_gender
3 participant_status
4 participant_type
5 latitude
6 longitude
7 gun_stolen
8 gun_type
9 incident_id
10 date_fk



In [4]:
rows=[]
for item in lines:
    row=item.split(",")
    rows.append(row)

rows

[['custody_id',
  'participant_age_group',
  'participant_gender',
  'participant_status',
  'participant_type',
  'latitude',
  'longitude',
  'gun_stolen',
  'gun_type',
  'incident_id',
  'date_fk\n'],
 ['10',
  'Adult 18+',
  'Male',
  'Arrested',
  'Suspect',
  '41.4455',
  '-82.1377',
  'Stolen',
  'Shotgun',
  '2',
  '0\n'],
 ['336',
  'Adult 18+',
  'Male',
  'Arrested',
  'Suspect',
  '29.8736',
  '-90.0568',
  'Stolen',
  'Shotgun',
  '56',
  '20\n'],
 ['1194',
  'Adult 18+',
  'Male',
  'Arrested',
  'Suspect',
  '39.9272',
  '-82.0073',
  'Stolen',
  'Shotgun',
  '209',
  '70\n'],
 ['1244',
  'Adult 18+',
  'Male',
  'Arrested',
  'Suspect',
  '36.1189',
  '-95.8429',
  'Stolen',
  'Shotgun',
  '219',
  '73\n'],
 ['1605',
  'Adult 18+',
  'Male',
  'Arrested',
  'Suspect',
  '30.6884',
  '-87.048',
  'Stolen',
  'Shotgun',
  '312',
  '86\n'],
 ['1650',
  'Adult 18+',
  'Male',
  'Arrested',
  'Suspect',
  '38.4333',
  '-84.3542',
  'Stolen',
  'Shotgun',
  '327',
  '86\n'],

## Geography

### Old version

In [ ]:
#pip install reverse_geocode
import reverse_geocode

In [ ]:
with open("geography.csv", "w", newline="") as file:
    writer=csv.writer(file)
    for item in rows:
        row=list((item[5], item[6]))
        writer.writerow(row)
        

In [ ]:
with open("Geography.csv", "r") as file:
    lines=file.readlines()

In [ ]:
coordinates=[]

for line in lines[1:]:
    c=line.split(",")
    coordinates.append(c)

In [ ]:
geo_list=[]

for coordinate in coordinates:
    value=[(float(coordinate[0]), float(coordinate[1]))]
    data=reverse_geocode.search(value)
    geo_list.append(data)

In [ ]:
len(geo_list)

In [ ]:
nums=[]

for element in lines[1:]:
    num=[float(x) for x in element.split(",")]
    nums.append(num)

In [ ]:
with open("geography_with_id.csv", "w", newline="") as file:
    writer=csv.writer(file)
    header=["latitude", "longitude", "city", "state, geo_id"]
    writer.writerow(header)

    i=0
    for item in nums:
        item.append(geo_list[i][0]["city"])
        item.append(geo_list[i][0]["country"])
        item.append(i)
        i+=1
        writer.writerow(item)

### New version

* We want to add the US cities and states to our database, since reverse-geocoder fails in this task. We want the state, not the entire country ('United States'). Moreover, reverse-geocoder finds some cities in Mexico and Canada which is wrong in our opinion.
* We find the city and state using an external csv availabel on the web, and then we compute the nearest city exploiting the coordinates (euclidean distance).

In [ ]:
with open('uscities.csv', 'r') as cities:
    # The biggest available dataset containing US cities for free
    cities_lines = cities.readlines()

    cities_list = []
    del cities_lines[5278]   # long: 'Monroe'???
    for e in cities_lines[1:]:
        e = e.strip().split(",")
        cities_list.append([e[0].strip('"'), e[2].strip('"'), e[3].strip('"'), float(e[6].strip('"')), float(e[7].strip('"'))])   # only useful data, already converted

print(cities_list[9])

In [ ]:
# check
lens = set()
for e in cities_list:
    lens.add(len(e))

print(max(lens))

In [ ]:
# We want to keep the same order as before to just upload geography without modifying custody

In [ ]:
with open("geography_with_id.csv", "r", encoding='utf-8') as geo:
    geo_lines=geo.readlines()

In [ ]:
import math

# We define two useful functions
def euclidean_distance(x1, y1, x2, y2):
    distance = math.sqrt(pow((x2 - x1), 2) + pow((y2 - y1), 2))
    return distance

In [ ]:
def search_city(lat, long, cities_list):
    best_match = 0
    state = 0
    best_dist = 10000.0

    for city in cities_list:
        #print(city)
        dist = euclidean_distance(lat, long, city[3], city[4])
        #print(dist)

        if dist < best_dist:
            best_dist = dist
            best_match = city[0]
            state = city[1]
    
    return best_match, best_dist, state

In [ ]:
# tests
best_match, best_dist, state = search_city(41.7594, -85.8815, cities_list)
best_match, best_dist, state

In [ ]:
# Final output
with open("geography_with_cities.csv", "w", newline="") as file:
    writer=csv.writer(file)

    header=["latitude", "longitude", "city", "state", "geo_id"]
    writer.writerow(header)

    lines_n = len(geo_lines)
    idx = 0
    for item in geo_lines[1:]:   # the first one is the header
        if idx % 1000 == 0:
            print('Progress:', (idx/lines_n)*100, '%, line:', idx)
        item = item.split(',')
        #print(item)
        best_match, best_dist, state = search_city(float(item[0]), float(item[1]), cities_list)
        
        row = list((float(item[0]), float(item[1]), best_match, state, int(item[4])))
        #print(row)
        writer.writerow(row)
        idx += 1

## Participant

In [ ]:
with open("partecipant_without_id.csv", "w", newline="") as file:
        writer=csv.writer(file)

        for item in rows:
            row=list((item[1], item[2], item[3]))
            writer.writerow(row)

In [ ]:
with open("partecipant_without_id.csv", "r") as file:
    partecipant_lines=file.readlines()

    lines_set = set(partecipant_lines)
    lines_set = lines_set.difference({'participant_age_group,participant_gender,participant_status\n'})
    print(lines_set)

    with open("partecipant.csv", "w", newline="") as file:
        writer=csv.writer(file)

        header=['participant_age_group, participant_status, participant_type, partecipant_id']
        writer.writerow(header)
        
        id = 0
        for item in lines_set:
            item = item[:-1].split(',')
            row=list((item[0], item[1], item[2], id))
            writer.writerow(row)
            id += 1

## Gun

In [ ]:
with open("gun.csv", "w", newline="") as file:
    writer=csv.writer(file)
    for item in rows:
        row=list((item[7], item[8]))
        writer.writerow(row)

In [6]:
# remove duplicates

with open("gun.csv", "r", newline="") as file:
    gun_reader = file.readlines()

print(gun_reader[0])

gun_stole,gun_type,gun_id



In [12]:
# remove duplicates

with open("gun_without_duplicates.csv", "w", newline="") as file:
    writer=csv.writer(file)

    header = ['gun_stole, gun_type, gun_id']
    writer.writerow(header)

    idx = 0
    for item in set(gun_reader[1:]):
        print(item)
        row = list(item[:-2].split(',')[:-1])
        row.append(idx)
        idx += 1
        print(row)
        writer.writerow(row)

Stolen,Handgun,4

['Stolen', 'Handgun', 0]
Unknown,380 Auto,40

['Unknown', '380 Auto', 1]
Stolen,380 Auto,2

['Stolen', '380 Auto', 2]
Stolen,22 LR,10

['Stolen', '22 LR', 3]
Unknown,32 Auto,74

['Unknown', '32 Auto', 4]
Not-stolen,45 Auto,25

['Not-stolen', '45 Auto', 5]
Not-stolen,20 gauge,52

['Not-stolen', '20 gauge', 6]
Not-stolen,10mm,61

['Not-stolen', '10mm', 7]
Stolen,44 Mag,27

['Stolen', '44 Mag', 8]
Stolen,12 gauge,29

['Stolen', '12 gauge', 9]
Stolen,357 Mag,21

['Stolen', '357 Mag', 10]
Not-stolen,Shotgun,34

['Not-stolen', 'Shotgun', 11]
Stolen,40 SW,14

['Stolen', '40 SW', 12]
Not-stolen,32 Auto,51

['Not-stolen', '32 Auto', 13]
Stolen,Shotgun,1

['Stolen', 'Shotgun', 14]
Unknown,20 gauge,75

['Unknown', '20 gauge', 15]
Unknown,Handgun,8

['Unknown', 'Handgun', 16]
Stolen,410 gauge,57

['Stolen', '410 gauge', 17]
Unknown,25 Auto,43

['Unknown', '25 Auto', 18]
Unknown,38 Spl,44

['Unknown', '38 Spl', 19]
Stolen,25 Auto,30

['Stolen', '25 Auto', 20]
Unknown,45 Auto,17

[

## Incident

In [ ]:
with open("incident.csv", "w", newline="") as file:
    writer=csv.writer(file)
    
    header = ['incident_id']
    rows_list = [header]
    
    for item in tqdm(rows):
        row=item[9]
        #print(row)
        rows_list.append(row)
        
    distincts = set(rows_list[1:])
    writer.writerow(rows_list[0])  # append the header
    for e in distincts:
        writer.writerow([e])
        
print(len(rows_list[1:]))
print(len(set(rows_list[1:])))

## Date

In [ ]:
with open("date.csv", "w", newline="") as file:
    writer=csv.writer(file)
    for item in rows:
        row=list((item[10]))
        writer.writerow(row)

In [ ]:
import xml.etree.ElementTree as ET

def convert_xml_to_csv(xml_file, csv_file):
    # XML file
    tree = ET.parse(xml_file)
    root = tree.getroot()

    with open(csv_file, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)

        # header
        writer.writerow(['Date'])

        for date_elem in root.iter('date'):
            date = date_elem.text

            writer.writerow([date])

    print("Complete")

# paths
# may come in handy in future
xml_file_path = 'dates.xml'
csv_file_path = 'dates_csv.csv'

# to CSV
convert_xml_to_csv(xml_file_path, csv_file_path)

In [ ]:
with open("dates_csv.csv", "r", newline="") as file:
    reader = file.readlines()
    #print(reader)
    
    with open("dates.csv", "w", newline="") as file1:
        writer=csv.writer(file1)
        rows=[]
        
        for i,e in enumerate(reader):
            rows.append(reader[i][:10])
        for e in rows:
            print(e)
            writer.writerow(e)

In [ ]:
# add the ID

with open("dates_csv.csv", "r", newline="") as file:
    reader = file.readlines()

    with open("dates.csv", "w", newline="") as file1:
        writer = csv.writer(file1)
        
        header = ['Date,Year,Month,Day,Date_id']   # Header
        rows = [header]
        
        id = 0
        for i, line in enumerate(reader[1:]):
            row = [line[:10]]  # Extract the first 10 characters and creates a list
            row.append(line[:4])
            row.append(line[5:7])
            row.append(line[8:10])
            row.append(id)
            id += 1
            
            rows.append(row)

        writer.writerows(rows)  # to CSV

## Custody

In [ ]:
# First we create all the dictionaries with the IDs

In [15]:
with open("dates.csv", "r", newline="") as file:
    dates_reader = file.readlines()

    dates_dict = {}
    for e in dates_reader:
        e = e.split(',')
        #print(e)
        dates_dict[(e[1], e[2], e[3])] = e[4][:-2]
    
print(dates_dict)

{('Year', 'Month', 'Day'): 'Date_id"', ('2013', '01', '01'): '0', ('2013', '01', '07'): '1', ('2013', '01', '19'): '2', ('2013', '01', '23'): '3', ('2013', '01', '25'): '4', ('2013', '01', '26'): '5', ('2013', '02', '02'): '6', ('2013', '02', '03'): '7', ('2013', '02', '09'): '8', ('2013', '02', '11'): '9', ('2013', '02', '19'): '10', ('2013', '03', '03'): '11', ('2013', '03', '04'): '12', ('2013', '03', '07'): '13', ('2013', '03', '10'): '14', ('2013', '03', '11'): '15', ('2013', '03', '13'): '16', ('2013', '03', '21'): '17', ('2013', '04', '18'): '18', ('2013', '04', '21'): '19', ('2013', '04', '22'): '20', ('2013', '04', '24'): '21', ('2013', '04', '25'): '22', ('2013', '04', '28'): '23', ('2013', '05', '02'): '24', ('2013', '05', '04'): '25', ('2013', '05', '11'): '26', ('2013', '05', '12'): '27', ('2013', '05', '13'): '28', ('2013', '05', '14'): '29', ('2013', '05', '16'): '30', ('2013', '05', '18'): '31', ('2013', '05', '19'): '32', ('2013', '05', '23'): '33', ('2013', '05', '25'

In [16]:
with open("partecipant.csv", "r", newline="") as file:
    partecipant_reader = file.readlines()

    partecipants_dict = {}
    for e in partecipant_reader:
        e = e.split(',')
        partecipants_dict[(e[0], e[1], e[2])] = e[3][:-2]

    print(partecipants_dict)

{('"participant_age_group', ' participant_status', ' participant_type'): ' partecipant_id"', ('Child 0-11', 'Male', 'Unharmed'): '0', ('Teen 12-17', 'Female', 'Injured'): '1', ('Teen 12-17', 'Female', 'Unharmed'): '2', ('Child 0-11', 'Male', 'Arrested'): '3', ('Teen 12-17', 'Male', 'Arrested'): '4', ('Adult 18+', 'Female', 'Arrested'): '5', ('Teen 12-17', 'Male', 'Unharmed'): '6', ('Teen 12-17', 'Female', 'Arrested'): '7', ('Adult 18+', 'Female', 'Injured'): '8', ('Adult 18+', 'Female', 'Killed'): '9', ('Adult 18+', 'Male', 'Injured'): '10', ('Adult 18+', 'Male', 'Arrested'): '11', ('Adult 18+', 'Male', 'Unharmed'): '12', ('Teen 12-17', 'Male', 'Injured'): '13', ('Teen 12-17', 'Female', 'Killed'): '14', ('Adult 18+', 'Male', 'Killed'): '15', ('Child 0-11', 'Female', 'Killed'): '16', ('Child 0-11', 'Female', 'Unharmed'): '17', ('Adult 18+', 'Female', 'Unharmed'): '18', ('Child 0-11', 'Female', 'Arrested'): '19', ('Teen 12-17', 'Male', 'Killed'): '20', ('Child 0-11', 'Male', 'Injured'): 

In [14]:
with open("gun_without_duplicates.csv", "r", newline="") as file:
    guns_reader = file.readlines()

    guns_dict = {}
    for e in guns_reader[1:]:
        e = e.split(',')
        #print(e)
        guns_dict[(e[0], e[1])] = e[2][:-2]

    print(guns_dict)

{('Stolen', 'Handgun'): '0', ('Unknown', '380 Auto'): '1', ('Stolen', '380 Auto'): '2', ('Stolen', '22 LR'): '3', ('Unknown', '32 Auto'): '4', ('Not-stolen', '45 Auto'): '5', ('Not-stolen', '20 gauge'): '6', ('Not-stolen', '10mm'): '7', ('Stolen', '44 Mag'): '8', ('Stolen', '12 gauge'): '9', ('Stolen', '357 Mag'): '10', ('Not-stolen', 'Shotgun'): '11', ('Stolen', '40 SW'): '12', ('Not-stolen', '32 Auto'): '13', ('Stolen', 'Shotgun'): '14', ('Unknown', '20 gauge'): '15', ('Unknown', 'Handgun'): '16', ('Stolen', '410 gauge'): '17', ('Unknown', '25 Auto'): '18', ('Unknown', '38 Spl'): '19', ('Stolen', '25 Auto'): '20', ('Unknown', '45 Auto'): '21', ('Unknown', 'Rifle'): '22', ('Stolen', '300 Win'): '23', ('Stolen', 'Rifle'): '24', ('Not-stolen', 'Unknown'): '25', ('Unknown', '44 Mag'): '26', ('Not-stolen', '410 gauge'): '27', ('Unknown', '16 gauge'): '28', ('Unknown', '357 Mag'): '29', ('Unknown', '223 Rem [AR-15]'): '30', ('Stolen', '30-30 Win'): '31', ('Not-stolen', '380 Auto'): '32', (

In [17]:
with open("geography_with_id.csv", "r", newline="", encoding='utf-8') as file:
    geo_reader = file.readlines()

    geo_dict = {}
    for e in geo_reader[1:]:
        e = e.split(',')

        lat = e[0]
        long = e[1]
        geo_dict[(lat, long)] = e[4][:-2]

    print(geo_dict)

{('41.7594', '-85.8815'): '0', ('41.4888', '-72.103'): '1', ('44.9556', '-93.1238'): '2', ('35.5055', '-97.5031'): '3', ('34.1339', '-118.147'): '4', ('47.6', '-122.332'): '5', ('30.8606', '-96.9722'): '6', ('41.7958', '-87.6695'): '7', ('33.7315', '-118.288'): '8', ('33.8376', '-118.351'): '9', ('33.5751', '-101.853'): '10', ('30.5188', '-91.1289'): '11', ('32.4184', '-90.1548'): '12', ('33.4117', '-91.0402'): '13', ('35.1092', '-89.9822'): '14', ('41.6728', '-72.7162'): '15', ('25.7399', '-80.2542'): '16', ('41.7914', '-72.6816'): '17', ('41.674', '-85.9725'): '18', ('39.8499', '-105.034'): '19', ('29.9928', '-90.0309'): '20', ('38.4986', '-80.0544'): '21', ('35.0664', '-78.8751'): '22', ('43.1144', '-89.3283'): '23', ('26.3707', '-81.8044'): '24', ('28.5035', '-82.336'): '25', ('42.4582', '-92.3557'): '26', ('44.0161', '-88.5826'): '27', ('34.9355', '-81.0474'): '28', ('39.5537', '-119.776'): '29', ('40.0284', '-82.948'): '30', ('40.8223', '-73.9497'): '31', ('30.2524', '-81.5379'):

In [18]:
# creates a mapping for the json files

files = ['dict_partecipant_age.json', 'dict_partecipant_status.json', 'dict_partecipant_type.json']
mapping = []

for file in files:
    # Opening JSON file
    f = open(file)
    data_age = json.load(f)
    
    # dict
    mapping.append(data_age)

    # Closing file
    f.close()

mapping

[{'Adult 18+': 1, 'Teen 12-17': 3, 'Child 0-11': 6},
 {'Arrested': 2, 'Killed': 4, 'Injured': 2, 'Unharmed': 1},
 {'Suspect': 1, 'Victim': 0}]

In [19]:
# final custody creation

with open("custody.csv", "w", newline="") as file:
    writer=csv.writer(file)
    # header
    header = ['custody_id, partecipant_id, gun_id, geo_id, date_id, incident_id, crime_gravity']
    writer.writerow(header)
    
    for item in rows[1:]:
        # crime gravity
        crime_gravity = mapping[0][item[1]] * mapping[1][item[3]] * mapping[2][item[4]]
        partecipant_id = partecipants_dict[(item[1], item[2], item[3])]
        gun_id = guns_dict[(item[7], item[8])]
        
        lat = item[5]
        long = item[6]
        coordinates = (lat, long)
        #print(str(item[5]), str(item[6]))
        
        geo_id = geo_dict[coordinates]

        date_id = item[10][:-1]
        
        row=list((item[0], partecipant_id, gun_id, geo_id, date_id, item[9], str(crime_gravity)))
        #print(row)
        writer.writerow(row)

In [10]:
rows[1:]

[['10',
  'Adult 18+',
  'Male',
  'Arrested',
  'Suspect',
  '41.4455',
  '-82.1377',
  'Stolen',
  'Shotgun',
  '2',
  '0\n'],
 ['336',
  'Adult 18+',
  'Male',
  'Arrested',
  'Suspect',
  '29.8736',
  '-90.0568',
  'Stolen',
  'Shotgun',
  '56',
  '20\n'],
 ['1194',
  'Adult 18+',
  'Male',
  'Arrested',
  'Suspect',
  '39.9272',
  '-82.0073',
  'Stolen',
  'Shotgun',
  '209',
  '70\n'],
 ['1244',
  'Adult 18+',
  'Male',
  'Arrested',
  'Suspect',
  '36.1189',
  '-95.8429',
  'Stolen',
  'Shotgun',
  '219',
  '73\n'],
 ['1605',
  'Adult 18+',
  'Male',
  'Arrested',
  'Suspect',
  '30.6884',
  '-87.048',
  'Stolen',
  'Shotgun',
  '312',
  '86\n'],
 ['1650',
  'Adult 18+',
  'Male',
  'Arrested',
  'Suspect',
  '38.4333',
  '-84.3542',
  'Stolen',
  'Shotgun',
  '327',
  '86\n'],
 ['2963',
  'Adult 18+',
  'Male',
  'Arrested',
  'Suspect',
  '45.5252',
  '-122.992',
  'Stolen',
  'Shotgun',
  '1032',
  '91\n'],
 ['3159',
  'Adult 18+',
  'Male',
  'Arrested',
  'Suspect',
  '36.8

## Export

### Gun

In [13]:
# connection
server = 'tcp:lds.di.unipi.it'
database = 'Group_ID_7_DB'
username = 'Group_ID_7'
password = 'N5XZKXE7'

# file path
csv_file_path = 'gun_without_duplicates.csv'
table_name = 'Gun'

# connection string
conn_str = f'DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}'
conn = pyodbc.connect(conn_str)

with open(csv_file_path, 'r', newline="") as csv_file:
    csv_reader = csv.reader(csv_file)
    
    header = next(csv_reader)
    
    # insert data into the table
    insert_query = f'INSERT INTO {table_name} VALUES (?,?,?)'
    for row in csv_reader:
        # DA MODIFICARE
        if row[2] != 'gun_id':
            row[2] = int(row[2])
            print(row)

            conn.execute(insert_query, row)
        else:
            continue

# IMPORTANT!
conn.commit()
conn.close()

['Stolen', 'Handgun', 0]
['Unknown', '380 Auto', 1]
['Stolen', '380 Auto', 2]
['Stolen', '22 LR', 3]
['Unknown', '32 Auto', 4]
['Not-stolen', '45 Auto', 5]
['Not-stolen', '20 gauge', 6]
['Not-stolen', '10mm', 7]
['Stolen', '44 Mag', 8]
['Stolen', '12 gauge', 9]
['Stolen', '357 Mag', 10]
['Not-stolen', 'Shotgun', 11]
['Stolen', '40 SW', 12]
['Not-stolen', '32 Auto', 13]
['Stolen', 'Shotgun', 14]
['Unknown', '20 gauge', 15]
['Unknown', 'Handgun', 16]
['Stolen', '410 gauge', 17]
['Unknown', '25 Auto', 18]
['Unknown', '38 Spl', 19]
['Stolen', '25 Auto', 20]
['Unknown', '45 Auto', 21]
['Unknown', 'Rifle', 22]
['Stolen', '300 Win', 23]
['Stolen', 'Rifle', 24]
['Not-stolen', 'Unknown', 25]
['Unknown', '44 Mag', 26]
['Not-stolen', '410 gauge', 27]
['Unknown', '16 gauge', 28]
['Unknown', '357 Mag', 29]
['Unknown', '223 Rem [AR-15]', 30]
['Stolen', '30-30 Win', 31]
['Not-stolen', '380 Auto', 32]
['Unknown', '30-30 Win', 33]
['Unknown', 'Other', 34]
['Unknown', '12 gauge', 35]
['Not-stolen', '300

### Date

In [ ]:
# connection details
server = 'tcp:lds.di.unipi.it'
database = 'Group_ID_7_DB'
username = 'Group_ID_7'
password = 'N5XZKXE7'

# CSV file path
csv_file_path = 'dates.csv'
table_name = 'Date'

# connection string
conn_str = f'DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}'
conn = pyodbc.connect(conn_str)

with open(csv_file_path, 'r', newline="") as csv_file:
    csv_reader = csv.reader(csv_file)
    
    header = next(csv_reader)
    
    # Insert data into the table
    insert_query = f'INSERT INTO {table_name} VALUES (?,?,?,?,?)'
    for row in csv_reader:
        if row[4] != 'Date_id':
            row[4] = int(row[4])
            print(row)

            conn.execute(insert_query, row)
        else:
            continue

conn.commit()
conn.close()

### Participants

In [ ]:
# connection
server = 'tcp:lds.di.unipi.it'
database = 'Group_ID_7_DB'
username = 'Group_ID_7'
password = 'N5XZKXE7'

# file path
csv_file_path = 'partecipant.csv'

table_name = 'Participant'

# connection string
conn_str = f'DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}'
conn = pyodbc.connect(conn_str)

with open(csv_file_path, 'r', newline="") as csv_file:
    csv_reader = csv.reader(csv_file)
    
    header = next(csv_reader)
    
    # insert data into the table
    insert_query = f'INSERT INTO {table_name} VALUES (?,?,?,?)'
    for row in csv_reader:
        # DA MODIFICARE
        if row[3] != 'partecipant_id':
            row[3] = int(row[3])
            print(row)

            conn.execute(insert_query, row)
        else:
            continue

# close the connection
conn.commit()
conn.close()

### Incident

In [ ]:
# connection
server = 'tcp:lds.di.unipi.it'
database = 'Group_ID_7_DB'
username = 'Group_ID_7'
password = 'N5XZKXE7'

# file path
csv_file_path = 'incident.csv'

table_name = 'Incident'

# connection string
conn_str = f'DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}'
conn = pyodbc.connect(conn_str)

with open(csv_file_path, 'r', newline="") as csv_file:
    csv_reader = csv.reader(csv_file)
    
    header = next(csv_reader)

    # insert data into the table
    insert_query = f'INSERT INTO {table_name} VALUES (?)'

    the_1000_rows = itertools.islice(csv_reader, 0, 20000)

    idx = 0
    for row in the_1000_rows:
        # DA MODIFICARE
        if row[0] != 'incident_id':
            row[0] = int(row[0])
            print(idx, ':', row)

            # Enable IDENTITY_INSERT for the table
            # we had problems with this option
            conn.execute("SET IDENTITY_INSERT Incident ON")
            conn.execute("INSERT INTO Incident (incident_id) VALUES (?)", row)
            idx += 1

            #conn.execute(insert_query, row)
        else:
            continue

# close the connection
conn.commit()
conn.close()

### Geography

In [ ]:
import itertools

In [ ]:
# connection
server = 'tcp:lds.di.unipi.it'
database = 'Group_ID_7_DB'
username = 'Group_ID_7'
password = 'N5XZKXE7'

# file path
csv_file_path = 'Geography_with_id.csv'

table_name = 'Geography'

# connection string
conn_str = f'DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}'
conn = pyodbc.connect(conn_str)

with open(csv_file_path, 'r', newline="", encoding='utf-8') as csv_file:
    csv_reader = csv.reader(csv_file)
    
    header = next(csv_reader)
    
    # splitting the operation in several parts, running time is about 2 hours
    the_1000_rows = itertools.islice(csv_reader, 70000, 96000)
    
    # insert data into the table
    # split into several partitions to let the program work in different sessions
    insert_query = f'INSERT INTO {table_name} VALUES (?,?,?,?,?)'
    for row in the_1000_rows:
        # DA MODIFICARE
        if row[4] != 'geo_id':
            row[4] = int(row[4])
            print(row)

            conn.execute(insert_query, row)
        else:
            continue

# close the connection
conn.commit()
conn.close()

### Custody

In [ ]:
# connection
server = 'tcp:lds.di.unipi.it'
database = 'Group_ID_7_DB'
username = 'Group_ID_7'
password = 'N5XZKXE7'

# file path
csv_file_path = 'custody.csv'

table_name = 'Custody'

# connection string
conn_str = f'DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}'
conn = pyodbc.connect(conn_str)

with open(csv_file_path, 'r', newline="") as csv_file:
    csv_reader = csv.reader(csv_file)
    
    header = next(csv_reader)
    
    # sam ething as for Geography
    the_1000_rows = itertools.islice(csv_reader, 2500, 60000)
    
    # insert data into the table
    # split into several partitions to let the program work in different sessions
    insert_query = f'INSERT INTO {table_name} VALUES (?,?,?,?,?,?,?)'
    
    idx = 0
    for row in the_1000_rows:
        # DA MODIFICARE
        if row[6] != 'crime_gravity':
            for i in range(7):
                row[i] = int(row[i])
            print(idx, ':', row)
            idx += 1

            conn.execute(insert_query, row)
        else:
            continue

In [ ]:
# close the connection
conn.commit()
conn.close()